In [ ]:
using Pkg
# wird weiter unten verwendet
environmentName = "04_Ex4"
Pkg.activate(joinpath(dirname(pwd()), "conf", environmentName))
Pkg.instantiate();

In [ ]:
using SparseArrays, Graphs, LinearAlgebra, Plots, GraphPlot;

In [ ]:
using JSON3

data = JSON3.read(read(joinpath(dirname(pwd()), "data", "02-networks", "graph.json")))
print(string(length(data)))

In [ ]:
pages = []
categories = []

# verwende nur die ersten 25 pages, damit der Graph nicht zu groß wird
for page in data[1:25]
    if haskey(page, :categories) && length(page.categories) > 0
        push!(pages, page.name)
        for category in page.categories
            push!(categories, category)
        end
    end
end

categories = unique(categories);

In [ ]:
pages

In [ ]:
categories

Der Graph G hat die Knotenmenge bestehend aus allen pages und allen categories. Eine Kante existiert nur zwischen zwei Knoten $u$ und $v$, wenn:
* $u$ ist eine page und $v$ ist eine category
* $u$ referenziert $v$

In [ ]:
using SparseArrays

n = length(pages) + length(categories)
V = 1:n
# diese Map übersetzt Namen von Seiten und Kategorien zu Knotennummern unter der Annahme, dass pages und categories disjoint sind
name_map = Dict()
# Gesamtanzahl an Knoten
A = zeros(Int, n, n)
E = []

# Seitennamen mit Knotennummern assoziieren
for i = 1:length(pages)
    name_map[data[i].name] = i
end
# Kategorien mit Knotennummern assoziieren, offset ist die Anzahl der Seiten
for j = 1:length(categories)
    name_map[string(categories[j])] = length(pages) + j
end

for i = 1:length(pages)
    page = data[i]
    for j = 1:length(page.categories)
        category = page.categories[j]
        A[i, name_map[category]] = A[name_map[category], i] = 1
    end
end

A = sparse(A)
A

In [ ]:
G = Graphs.Graph(A)

In [ ]:
d = Graphs.degree(G)
not0 = findall(d .!= 0)

In [ ]:
#eliminating isolated vertices from G
A = A[not0,not0]
d = d[not0]

n = size(A)[1]; 
G = Graphs.Graph(A)
sparse(A)

In [ ]:
#building the Laplace matrix

L = zeros(n, n)
for i in 1:n
    for j in 1:n
        if i == j
            L[i,j] = 1
        elseif A[i,j] == 1
            L[i,j] = - 1 / sqrt(d[i] * d[j])
        end
    end
end
L;

In [ ]:
#computing and plotting the spectrum

E = eigen(L)
λ = E.values
P = plot(λ, label = "The spectrum of G",
             linewidth = 3,
             xlab = "i",
             ylab = "lambda_i")

In [ ]:
revλ = sort(2 .-λ)

In [ ]:
#investigating symmetry of the spectrum around 1 (check Sheet 5!)
count(abs.(λ .- revλ) .< 10^(-14)), n

In [ ]:
#studying the biggest connected component H
C = connected_components(G)
k = findmax(length.(C))
I = C[k[2]]
H = Graph(A[I, I])

In [ ]:
layout = (zeros(n), zeros(n))
height = max(length(pages), length(categories))
for i = 1:length(pages)
        layout[1][i] = 10
        layout[2][i] = i / length(pages) * height
end
for j = 1:length(categories)
        layout[1][j + length(pages)] = 50
        layout[2][j + length(pages)] = j / length(categories) * height
end
layout_H = (layout[1][I], layout[2][I])
P2 = gplot(H, layout_H...;
        nodelabeldist=2,
        nodefillc = colorant"steelblue"
)

In [ ]:
L_H = L[I,I]
E_H = eigen(L_H)
λ_H = E_H.values
plot(λ_H, legend = false, linewidth = 3)

In [ ]:
using Compose, Cairo, Fontconfig, IJulia, FileIO, Images

#finding the bipartition of H via the eigenvector associated with 2
u = E_H.vectors[:,end]
cols = []
for uᵢ in u
    if abs(uᵢ) > 1e-12 
        if uᵢ < 0
            push!(cols, colorant"steelblue")
        else
            push!(cols, colorant"indianred")
        end
    else
        push!(cols, colorant"orange")
    end
end
P3 = gplot(H, layout_H...;
        nodelabeldist=2,
        nodefillc = cols
)
# wir speichern das Bild und plotten dann das nachgeladene Bild,
# weil jupyter teilweise Probleme damit hat gefärbte Graphen anzuzeigen, die viele Knoten enthalten
imagePath = joinpath(dirname(pwd()), "img", environmentName, "bipartiton_of_H.png")
draw(PNG(imagePath, 16cm, 16cm), P3)
IJulia.display(load(imagePath))